In [1]:
from google.colab import files
import pandas as pd

# Pregunta a l'usuari que pugi un arxiu Excel
uploaded = files.upload()

# Obtenir el nom de l'arxiu pujat
nom_arxiu = list(uploaded.keys())[0]

# Carregar l'arxiu Excel a partir de la fila 14
df = pd.read_excel(nom_arxiu, skiprows=13)

Saving Inventaridedretsimmaterials-Datesdevigència (25).xlsx to Inventaridedretsimmaterials-Datesdevigència (25).xlsx


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [2]:
import datetime

avui = pd.to_datetime(datetime.date.today())

df['data'] = pd.to_datetime(df['Data fi vigència'], errors='coerce')

In [3]:
# Suposem que ja has carregat el teu DataFrame df

# Filtre 1: Estat = "Alta"
filtre_estat = df['Estat'] == 'Alta'

# Filtre 2: Catàleg = "Marca"
filtre_catalog = df['Catàleg'] == 'Marca'

# Filtre 3: Registre d'inscripció = "Oficina de Propietat Intel·lectual de la Unió Europea (EUIPO)"
filtre_registre = df['Registre d\'inscripció'] == 'Oficina de Propietat Intel·lectual de la Unió Europea (EUIPO)'

# Filtre 4: Data Fi vigència anterior a la data d'avui
filtre_vigencia = df['Data fi vigència'] < avui

# Aplica els filtres
df_filtrat = df[filtre_estat & filtre_catalog & filtre_registre & filtre_vigencia]

# Mostra el DataFrame filtrat
print(len(df_filtrat))

35


In [4]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 4.2 MB/s eta 0:00:00


In [8]:
!pip install selenium

In [9]:
!apt-get update
!apt install chromium-chromedriver -y

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 https://cli.github.com/packages stable InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,526 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,969 

In [13]:
import sys
import chromedriver_autoinstaller
from selenium import webdriver

# Instal·la el chromedriver automàticament
chromedriver_autoinstaller.install()

# Opcions per al WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Executa en mode sense interfície gràfica
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Crea la instància del WebDriver
driver = webdriver.Chrome(options=chrome_options)

In [25]:
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
import time
import re
from bs4 import BeautifulSoup
import json

def euipo(codi):

  url = "https://euipo.europa.eu/eSearch/#details/trademarks/" +codi.lstrip('0').zfill(9)
  print(url)

  driver.get(url)

  #while True:
  while True:
      try:
          # Espera fins que l'element amb la classe "box-content detailsBoxBody" estigui present
          element = WebDriverWait(driver, 10).until(
              EC.presence_of_element_located((By.CLASS_NAME, "box-content detailsBoxBody"))
          )

          # Aquí pots continuar amb la lògica després que l'element estigui present
          print("Element carregat:", element.text)

      except:
          break

  data_dict = {}

  # Assumim que ja tens el codi HTML emmagatzemat a la variable 'web'
  html_content = driver.page_source

  # Parseja l'HTML amb BeautifulSoup
  soup = BeautifulSoup(html_content, 'html.parser')

  # Obté els elements dl-horizontal i converteix-los en un diccionari
  dl_elements = soup.find_all('dl', class_='dl-horizontal dl-divided')

  # Itera a través dels elements dl
  for dl_index, dl_element in enumerate(dl_elements):
      dt_elements = dl_element.find_all('dt')
      dd_elements = dl_element.find_all('dd')

      # Itera a través dels elements dt i dd i emmagatzema la informació al diccionari
      for dt_index, (dt, dd) in enumerate(zip(dt_elements, dd_elements)):
          field_name = f"{dt.text.strip()}_{dl_index}_{dt_index}"
          data_dict[field_name] = dd.text.strip()

  # Verifica si 'Filing number_0_1' està al diccionari
  if 'Filing number_0_1' not in data_dict:
      # Si no està present, crea la clau amb el valor "codi"
      data_dict['Filing number_0_1'] = codi.lstrip('0').zfill(9)

  # Elimina els caràcters '\n' i els espais múltiples del diccionari
  cleaned_data_dict = {key: re.sub(r'\s+', ' ', value.replace('\n', '').strip()) for key, value in data_dict.items()}

  try:
    # Obté l'atribut 'src' de la etiqueta <img> dins de la classe 'detailsTrademarkImage'
    img_tag = soup.find('p', class_='detailsTrademarkImage').find('img')
    img_src = img_tag['src'] if img_tag else None

    # Construeix la URL completa
    full_url = f"https://euipo.europa.eu{img_src}" if img_src else None  # Canvia 'https://example.com' per la teva URL base
  except:
    full_url = ""

  time.sleep(3)

  # Imprimeix el JSON netejat
  return cleaned_data_dict, full_url

In [17]:
# Exemple d'ús

print(euipo("12235065"))
#df_filtrat["Número d'inventari"][df_filtrat["Codi registre"] == x].iloc[0]

https://euipo.europa.eu/eSearch/#details/trademarks/012235065
({'Name_0_0': 'CATALONIA CLUSTERS', 'Filing number_0_1': '012235065', 'Basis_0_2': 'EUTM', 'Date of receipt_0_3': '18/10/2013', 'Type_0_4': 'Figurative', 'Nature_0_5': 'Individual', 'Nice classes_0_6': '35, 36, 41, 42 ( Nice Classification )', 'Vienna Classification_0_7': '24.01.03, 24.01.25, 25.01.25, 26.04.02, 26.04.05, 26.04.22, 26.04.98, 29.01.01 ( Vienna Classification )', 'Filing date_1_0': '18/10/2013', 'Registration date_1_1': '06/03/2014', 'Expiry date_1_2': '18/10/2033', 'Designation date_1_3': '', 'Filing language_1_4': 'Spanish', 'Second language_1_5': 'English', 'Application reference_1_6': 'INT7059', 'Trade mark status_1_7': 'Registered', 'Acquired distinctiveness_1_8': 'No', 'ID_2_0': '420124', 'Organisation_2_1': "AGENCIA PER A LA COMPETITIVITAT DE L'EMPRESA", 'Legal status_2_2': 'Legal entity', 'ID_3_0': '22326', 'Organisation_3_1': 'n/a', 'Legal status_3_2': 'Legal person', 'Type_3_3': 'Association'}, 'http

In [26]:
json_final = {"euipo": []}
imatges = []
l = len(df_filtrat)
i = 1

# Itera a través de la columna "Codi registre" i obté la primera fila
for x in df_filtrat["Codi registre"]:
  print(i, "de", l)
  codi = str(int(float(x)))
  try:
    seccio = df_filtrat["Número d'inventari"][df_filtrat["Codi registre"] == x].iloc[0]
    resultat_iteracio, url = euipo(codi)
    json_final["euipo"].append(resultat_iteracio)
    json_final["euipo"][i-1]["seccio"] = str(seccio)
    imatges.append([codi, url])
    i += 1
  except Exception as e:
    json_final["euipo"].append({"codi": codi, "seccio": "Codi incorrecte"})
    imatges.append([codi, ""])

# Mostra el JSON final
print(json_final)
print(imatges)

1 de 35
https://euipo.europa.eu/eSearch/#details/trademarks/012018073
2 de 35
https://euipo.europa.eu/eSearch/#details/trademarks/014131924
3 de 35
https://euipo.europa.eu/eSearch/#details/trademarks/014173165
4 de 35
https://euipo.europa.eu/eSearch/#details/trademarks/004665469
5 de 35
https://euipo.europa.eu/eSearch/#details/trademarks/014748065
{'euipo': [{'Name_0_0': "SOC Servei d'Ocupació de Catalunya", 'Filing number_0_1': '004665469', 'Basis_0_2': 'EUTM', 'Date of receipt_0_3': '04/10/2005', 'Type_0_4': 'Figurative', 'Nature_0_5': 'Individual', 'Nice classes_0_6': '9, 16, 25, 28, 35, 36, 38, 41, 42 ( Nice Classification )', 'Vienna Classification_0_7': '24.01.03, 24.01.15 ( Vienna Classification )', 'Filing date_1_0': '04/10/2005', 'Registration date_1_1': '18/10/2006', 'Expiry date_1_2': '04/10/2035', 'Designation date_1_3': '', 'Filing language_1_4': 'Spanish', 'Second language_1_5': 'English', 'Application reference_1_6': '1705', 'Trade mark status_1_7': 'Registered', 'Acquir

In [27]:
euipo_final = pd.DataFrame.from_dict(json_final["euipo"], orient='columns')
columna_a_moure = euipo_final.pop('seccio')  # Extreure la columna de DataFrame i guardar-la en una variable
euipo_final.insert(0, 'seccio', columna_a_moure)  # Inserir la columna a la primera posició del DataFrame

euipo_final

,seccio,Name_0_0,Filing number_0_1,Basis_0_2,Date of receipt_0_3,Type_0_4,Nature_0_5,Nice classes_0_6,Vienna Classification_0_7,Filing date_1_0,...,Application reference_1_6,Trade mark status_1_7,Acquired distinctiveness_1_8,ID_2_0,Organisation_2_1,Legal status_2_2,ID_3_0,Organisation_3_1,Legal status_3_2,Type_3_3
0,163,SOC Servei d'Ocupació de Catalunya,004665469,EUTM,04/10/2005,Figurative,Individual,"9, 16, 25, 28, 35, 36, 38, 41, 42 ( Nice Class...","24.01.03, 24.01.15 ( Vienna Classification )",04/10/2005,...,1705,Registered,No,237059,SERVEI PÚBLIC D'OCUPACIÓ DE CATALUNYA,Legal entity,72637,n/a,Legal person,Association
1,178,PREMI PROTECCIÓ DE DADES EN EL DISSENY APDCAT ...,012018073,EUTM,26/07/2013,Figurative,Individual,"16, 41, 42 ( Nice Classification )","26.04.02, 26.04.08, 26.04.19, 26.04.22 ( Vienn...",26/07/2013,...,1784-05,Registration expired,No,436239,AUTORITAT CATALANA DE PROTECCIÓ DE DADES,Legal entity,72637,n/a,Legal person,Association
2,179,REVISTA DE LLENGUA I DRET RLD JOURNAL OF LANGU...,014131924,EUTM,26/05/2015,Figurative,Individual,9 ( Nice Classification ),"26.04.02, 26.04.05, 26.04.18, 26.04.99, 26.11....",26/05/2015,...,M79136EU00,Registered,No,697493,ESCOLA D'ADMINISTRACIÓ PÚBLICA DE CATALUNYA,Legal entity,15051,n/a,Legal person,Association
3,4624,REVISTA DE LLENGUA I DRET - JOURNAL OF LANGUAG...,014173165,EUTM,28/05/2015,Figurative,Individual,9 ( Nice Classification ),"25.05.02, 26.01.03, 26.04.02, 26.04.05, 26.04....",28/05/2015,...,M79138EU00,Registered,No,697493,ESCOLA D'ADMINISTRACIÓ PÚBLICA DE CATALUNYA,Legal entity,15051,n/a,Legal person,Association
4,4626,REVISTA DE LLENGUA I DRET - JOURNAL OF LANGUAG...,014173165,EUTM,28/05/2015,Figurative,Individual,9 ( Nice Classification ),"25.05.02, 26.01.03, 26.04.02, 26.04.05, 26.04....",28/05/2015,...,M79138EU00,Registered,No,697493,ESCOLA D'ADMINISTRACIÓ PÚBLICA DE CATALUNYA,Legal entity,15051,n/a,Legal person,Association


In [22]:
imatges = pd.DataFrame(imatges)
imatges

,0,1
0,12018073,
1,14131924,
2,14173165,
3,4665469,
4,14748065,https://euipo.europa.eu/copla/image/CJ4JX4FZVC...


In [23]:
import datetime

avui = datetime.datetime.now()
avui = str(avui).split(" ")[0].replace("-", "")

In [24]:
import pandas as pd
import xlsxwriter
import datetime
from google.colab import files

avui = datetime.datetime.now()
avui = str(avui).split(" ")[0].replace("-", "")

# Crear l'arxiu d'Excel i afegir-hi les tres taules en pestanyes diferents
with pd.ExcelWriter(avui+"_euipo_marques.xlsx", engine='xlsxwriter') as writer:
    euipo_final.to_excel(writer, sheet_name='Marques', index=False)
    imatges.to_excel(writer, sheet_name='Imatges', index=False)

files.download(avui+"_euipo_marques.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''
codi = "12235065"

url = "https://euipo.europa.eu/eSearch/#details/trademarks/" +codi.lstrip('0').zfill(9)

driver.get(url)

#while True:
while True:
    try:
        # Espera fins que l'element amb la classe "box-content detailsBoxBody" estigui present
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "box-content detailsBoxBody"))
        )

        # Aquí pots continuar amb la lògica després que l'element estigui present
        print("Element carregat:", element.text)

    except:
        break

# Assumim que ja tens el codi HTML emmagatzemat a la variable 'web'
html_content = driver.page_source

# Parseja l'HTML amb BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Assumim que ja tens el codi HTML emmagatzemat a la variable 'web'
html_content = driver.page_source

# Parseja l'HTML amb BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Obté els elements dl-horizontal i converteix-los en un diccionari
data_dict = {}
dl_elements = soup.find_all('dl', class_='dl-horizontal dl-divided')

# Itera a través dels elements dl
for dl_index, dl_element in enumerate(dl_elements):
    dt_elements = dl_element.find_all('dt')
    dd_elements = dl_element.find_all('dd')

    # Itera a través dels elements dt i dd i emmagatzema la informació al diccionari
    for dt_index, (dt, dd) in enumerate(zip(dt_elements, dd_elements)):
        field_name = f"{dt.text.strip()}_{dl_index}_{dt_index}"
        data_dict[field_name] = dd.text.strip()

# Elimina els caràcters '\n' i els espais múltiples del diccionari
cleaned_data_dict = {key: re.sub(r'\s+', ' ', value.replace('\n', '').strip()) for key, value in data_dict.items()}

# Mostra el diccionari amb les dades extretes
print(cleaned_data_dict)
'''

'\ncodi = "12235065"\n\nurl = "https://euipo.europa.eu/eSearch/#details/trademarks/" +codi.lstrip(\'0\').zfill(9)\n\ndriver.get(url)\n\n#while True:\nwhile True:\n    try:\n        # Espera fins que l\'element amb la classe "box-content detailsBoxBody" estigui present\n        element = WebDriverWait(driver, 10).until(\n            EC.presence_of_element_located((By.CLASS_NAME, "box-content detailsBoxBody"))\n        )\n\n        # Aquí pots continuar amb la lògica després que l\'element estigui present\n        print("Element carregat:", element.text)\n\n    except:\n        break\n\n# Assumim que ja tens el codi HTML emmagatzemat a la variable \'web\'\nhtml_content = driver.page_source\n\n# Parseja l\'HTML amb BeautifulSoup\nsoup = BeautifulSoup(html_content, \'html.parser\')\n\n# Assumim que ja tens el codi HTML emmagatzemat a la variable \'web\'\nhtml_content = driver.page_source\n\n# Parseja l\'HTML amb BeautifulSoup\nsoup = BeautifulSoup(html_content, \'html.parser\')\n\n# Obté e